In [3]:
import torch
import torch.nn as nn


class BigModel(nn.Module):
    def __init__(self):
        super(BigModel, self).__init__()
        self.fc = nn.Linear(500, 500)  # 500 * 500 = 250,000 parameters

    def forward(self, x):
        return self.fc(x)

生成一个1MB的模型，需要大约250,000个参数，因为1MB = 1024 * 1024字节，所以大约250,000个参数

In [4]:
# 创建模型实例
model = BigModel()

# 保存模型
torch.save(model.state_dict(), 'big_model.pth')

In [5]:
# 检查模型大小
import os
print(f'Model size: {os.path.getsize("big_model.pth") / 1024 / 1024} MB')

Model size: 0.9570655822753906 MB


在这个例子中，我们首先保存模型的参数（model.state_dict()），然后训练模型并保存新的参数。我们计算参数的更新（新的参数减去旧的参数），并保存这些更新。当我们需要加载模型时，我们加载旧的参数和更新，然后应用这些更新（旧的参数加上更新）。
这个例子只是一个基本的示例，实际情况可能会更复杂 🐶

In [6]:
import torch.optim as optim

# 创建优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 创建损失函数
criterion = nn.MSELoss()

# 创建一个简单的数据集
inputs = torch.randn(10, 500)
targets = torch.randn(10, 500)

# 保存旧的状态字典
old_state_dict = {name: param.data.clone() for name, param in model.state_dict().items()}

# 训练模型
model.train()
optimizer.zero_grad()
outputs = model(inputs)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

# 保存新的状态字典
new_state_dict = {name: param.data for name, param in model.state_dict().items()}

# 计算参数的更新
updates = {name: new_state_dict[name] - old_state_dict[name] for name in new_state_dict}

# 保存模型和参数更新
torch.save(new_state_dict, 'new_model.pth')

得到参数更新的update模型和最后结果的模型

In [7]:
# 将updates中的一维矩阵转换为二维
for key in updates:
    if updates[key].ndim == 1:
        updates[key] = updates[key].view(1, -1) 

for key in updates:
    print(key, updates[key].shape)

fc.weight torch.Size([500, 500])
fc.bias torch.Size([1, 500])


将一维的tensor的转换为2维, 方便之后使用奇异值分解（SVD）求的低秩近似。

In [8]:
import numpy as np

def low_rank_approximation(matrix, k):
    """返回矩阵的低秩近似，k为奇异值个数"""
    U, s, Vh = np.linalg.svd(matrix, full_matrices=False)
    U_k = U[:, :k]
    s_k = s[:k]
    Vh_k = Vh[:k, :]
    return U_k @ np.diag(s_k) @ Vh_k

# 对每个参数应用低秩近似
k = 10  # 你可以根据需要设置k的值
updates_approx = {}
for name, update in updates.items():
    # 检查更新是否为二维数组
    if len(update.shape) == 2:
        # 首先将Tensor转换为numpy数组，进行SVD，然后再转回Tensor
        update_numpy = update.cpu().numpy()
        update_approx_numpy = low_rank_approximation(update_numpy, k)
        updates_approx[name] = torch.from_numpy(update_approx_numpy).to(update.device)
    else:
        # 对于一维数组，我们只是简单地复制更新
        updates_approx[name] = update.clone()

torch.save(updates_approx, 'updates.pth')

使用奇异值分解（SVD）来获取一个矩阵的低秩近似。
🐶 但不起效果。
应该将将一些全连接层（或卷积层）替换为低秩版本。这涉及模型压缩或模型剪枝，难度颇大，黑客松上暂时不考虑。

In [9]:
# 检查模型大小
import os
print(f'Model size: {os.path.getsize("updates.pth") / 1024 / 1024} MB')

Model size: 0.9568710327148438 MB
